In [85]:
import numpy as np
import pandas as pd
import os

root_path = '/Users/shijunshen/Documents/Code/dataset/RL/AutonomousGreenhouseChallenge_edition2/IUACAAS'

# Condition

In [86]:
greenhouse_path = os.path.join(root_path, 'GreenhouseClimate.csv')

greenhouse_df = pd.read_csv(greenhouse_path, low_memory=False)
# Tair（温度）, Rhair（湿度）, CO2air, EC_drain_PC, pH_drain_PC, water_sup
greenhouse_df = greenhouse_df[['%time', 'Tair', 'Rhair', 'CO2air', 'EC_drain_PC', 'pH_drain_PC', 'water_sup']]

greenhouse_df.head()

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.00000,21.2,64.0,445,2.1,7.1,2.0
1,43815.00347,22.0,61.7,490,2.1,7.1,4.0
2,43815.00694,21.3,67.5,511,2.1,7.1,4.0
3,43815.01042,21.6,67.9,510,2.1,7.1,4.0
4,43815.01389,21.6,65.4,501,2.1,7.1,4.0


In [87]:
# get Avg value for each day
greenhouse_df = greenhouse_df.apply(pd.to_numeric, errors='coerce')

greenhouse_dict = {}
temp_dict = {}

for column in greenhouse_df.columns:
    greenhouse_dict[column] = []
    temp_dict[column] = []

current_day = int(greenhouse_df['%time'][0])
temp_df = pd.DataFrame(columns=greenhouse_df.columns)
greenhouse_avgday_df = pd.DataFrame(columns=greenhouse_df.columns)
water_sup = 0
for index, row in greenhouse_df.iterrows():
    if int(row['%time']) == current_day:
        temp_df = pd.concat([temp_df, row.to_frame().T])
        if water_sup < row['water_sup']:
            water_sup = row['water_sup']
    else:
        temp_df = temp_df.mean(axis=0)
        temp_df['%time'] = current_day
        temp_df['water_sup'] = water_sup
        greenhouse_avgday_df = pd.concat([greenhouse_avgday_df, temp_df.to_frame().T])
        temp_df = pd.DataFrame(columns=greenhouse_df.columns)
        current_day = int(row['%time'])
        water_sup = 0
greenhouse_avgday_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.0,19.660069,68.669444,562.312500,2.100000,7.100000,6.0
0,43816.0,16.779443,80.286411,585.216028,2.100000,7.100000,0.0
0,43817.0,17.407666,74.840418,661.073171,2.100000,7.100000,0.0
0,43818.0,17.259930,79.470035,652.797909,2.100000,7.100000,0.0
0,43819.0,18.479094,72.503484,627.404181,2.050871,7.203833,5.0
...,...,...,...,...,...,...,...
0,43976.0,23.934495,77.389547,449.710801,4.216725,5.801045,48.0
0,43977.0,23.992334,73.883275,478.522648,4.600697,5.778397,42.0
0,43978.0,24.197561,76.410453,452.494774,5.144948,5.744948,45.0
0,43979.0,26.700697,79.047387,439.909408,5.555052,5.672125,48.0


In [88]:
# Data cleaning (set Nan value to the previous value)
greenhouse_avgday_df = greenhouse_avgday_df.apply(pd.to_numeric, errors='coerce')
greenhouse_avgday_df.fillna(method='ffill', inplace=True)
greenhouse_avgday_df.fillna(method='bfill', inplace=True)
greenhouse_avgday_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43815.0,19.660069,68.669444,562.312500,2.100000,7.100000,6.0
0,43816.0,16.779443,80.286411,585.216028,2.100000,7.100000,0.0
0,43817.0,17.407666,74.840418,661.073171,2.100000,7.100000,0.0
0,43818.0,17.259930,79.470035,652.797909,2.100000,7.100000,0.0
0,43819.0,18.479094,72.503484,627.404181,2.050871,7.203833,5.0
...,...,...,...,...,...,...,...
0,43976.0,23.934495,77.389547,449.710801,4.216725,5.801045,48.0
0,43977.0,23.992334,73.883275,478.522648,4.600697,5.778397,42.0
0,43978.0,24.197561,76.410453,452.494774,5.144948,5.744948,45.0
0,43979.0,26.700697,79.047387,439.909408,5.555052,5.672125,48.0


# Result

In [89]:
# quality

quality_path = os.path.join(root_path, 'TomQuality.csv')

quality_df = pd.read_csv(quality_path, low_memory=False)

quality_df = quality_df.apply(pd.to_numeric, errors='coerce')
quality_df.fillna(method='ffill', inplace=True)
quality_df.fillna(method='bfill', inplace=True)
quality_df

,%time,Flavour,TSS,Acid,%Juice,Bite,Weight,DMC_fruit
0,43880,74,8.0,13.6,60,177,9.63,9.28
1,43894,74,8.1,14.9,62,244,9.53,9.28
2,43908,74,8.3,14.4,57,308,9.67,9.28
3,43922,80,9.4,14.3,58,254,9.70,9.28
4,43936,75,8.9,14.6,50,312,8.70,9.09
5,43950,75,8.5,12.6,64,260,10.10,8.76
6,43964,78,9.0,13.3,63,208,11.07,8.36
7,43980,74,8.2,14.3,60,232,9.90,9.51


# Combine Result and Condition

In [90]:
# set the first timestamp of result as the beginning in both condition and result
temp_sum = pd.DataFrame(columns=greenhouse_avgday_df.columns)
greenhouse_avg_interval_df = pd.DataFrame(columns=greenhouse_df.columns)

for index, row in quality_df.iterrows():
    # print(index, row)
    if index == 0:
        continue
    left_time = quality_df['%time'][index-1]
    right_time = quality_df['%time'][index]
    for i, r in greenhouse_avgday_df.iterrows():
        if r['%time'] < left_time:
            continue
        if r['%time'] >= right_time:
            temp_sum = temp_sum.mean(axis=0)
            temp_sum['%time'] = left_time
            greenhouse_avg_interval_df = pd.concat([greenhouse_avg_interval_df, temp_sum.to_frame().T])
            temp_sum = pd.DataFrame(columns=greenhouse_avgday_df.columns)
            # add last timestamp info
            if index == quality_df.shape[0] - 1 and r['%time'] == right_time:
                greenhouse_avg_interval_df = pd.concat([greenhouse_avg_interval_df, r.to_frame().T])
            break
        temp_sum = pd.concat([temp_sum, r.to_frame().T])
    
greenhouse_avg_interval_df

,%time,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43880.0,21.089036,86.098420,701.752109,5.356422,5.003402,43.428571
0,43894.0,21.898507,87.953360,688.666501,5.046043,4.555500,43.142857
0,43908.0,22.056609,86.734262,629.346732,5.717942,4.312231,50.571429
0,43922.0,21.650224,86.134719,596.525386,6.401568,4.102489,59.428571
0,43936.0,21.590418,84.488079,573.891986,5.562892,4.226008,69.000000
0,43950.0,21.516949,82.651891,576.615480,4.975411,5.527476,61.928571
0,43964.0,22.812587,74.948127,493.383057,4.871254,5.701677,52.312500
0,43980.0,26.258188,73.559233,447.668990,5.867247,5.590244,48.000000


In [91]:
merged_df = pd.merge(quality_df, greenhouse_avg_interval_df, on='%time', how='outer')
merged_df

,%time,Flavour,TSS,Acid,%Juice,Bite,Weight,DMC_fruit,Tair,Rhair,CO2air,EC_drain_PC,pH_drain_PC,water_sup
0,43880,74,8.0,13.6,60,177,9.63,9.28,21.089036,86.098420,701.752109,5.356422,5.003402,43.428571
1,43894,74,8.1,14.9,62,244,9.53,9.28,21.898507,87.953360,688.666501,5.046043,4.555500,43.142857
2,43908,74,8.3,14.4,57,308,9.67,9.28,22.056609,86.734262,629.346732,5.717942,4.312231,50.571429
3,43922,80,9.4,14.3,58,254,9.70,9.28,21.650224,86.134719,596.525386,6.401568,4.102489,59.428571
4,43936,75,8.9,14.6,50,312,8.70,9.09,21.590418,84.488079,573.891986,5.562892,4.226008,69.000000
5,43950,75,8.5,12.6,64,260,10.10,8.76,21.516949,82.651891,576.615480,4.975411,5.527476,61.928571
6,43964,78,9.0,13.3,63,208,11.07,8.36,22.812587,74.948127,493.383057,4.871254,5.701677,52.312500
7,43980,74,8.2,14.3,60,232,9.90,9.51,26.258188,73.559233,447.668990,5.867247,5.590244,48.000000
